# Import Libraries

In [1]:
!pip install -q -U bitsandbytes
!pip install -q -U datasets
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q -U loralib
!pip install -q -U einops
!pip install -q -U googletrans==3.1.0a0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 21.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 2.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━

In [2]:
import json
import os
import bitsandbytes as bnb
import torch
import torch.nn as nn
import transformers

from googletrans import Translator
from pprint import pprint
from datasets import load_dataset
from huggingface_hub import notebook_login
from peft import (
    LoraConfig,
    PeftConfig,
    PeftModel,
    get_peft_model,
    prepare_model_for_kbit_training
)
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig
)

os.environ['CUDA_VISIBLE_DEVICES'] = '0'

# Load the pretrained LLM

In [3]:
MODEL_NAME = 'vilm/vinallama-7b-chat'

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map='auto',
    trust_remote_code=True,
    quantization_config=bnb_config
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=[
        'q_proj',
        'up_proj',
        'o_proj',
        'k_proj',
        'down_proj',
        'gate_proj',
        'v_proj'
    ],
    lora_dropout=0.05,
    bias='none',
    task_type='CAUSAL_LM'
)

model = get_peft_model(model, config)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/709 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.22k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.67M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/557 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
def print_trainable_parameters(model):
    trainable_params = 0
    all_params = 0
    for _, param in model.named_parameters():
        all_params += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()

    print(f'Trainable params: {trainable_params} || All params: {all_params} || Trainables%: {100 * trainable_params / all_params}')

In [5]:
print_trainable_parameters(model)

Trainable params: 39976960 || All params: 3657576448 || Trainables%: 1.092990415056391


# Test pretrained model performance

In [6]:
generation_config = model.generation_config
generation_config.max_new_tokens = 200
generation_config.temperature = 0.7
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

In [7]:
prompt = '''
<|im_start|>system
Bạn là một trợ lí AI hữu ích. Hãy trả lời người dùng một cách chính xác.
<|im_end|>
<|im_start|>user
Viết một hàm tính tổng hai số trong python
<|im_end|>
<|im_start|>assistant
'''.strip()

In [8]:
def predict_result(prompt):
    encoding = tokenizer(prompt, return_tensors='pt').to(device)
    with torch.inference_mode():
        outputs = model.generate(
            input_ids=encoding.input_ids,
            attention_mask=encoding.attention_mask,
            generation_config=generation_config
        )

    print(tokenizer.decode(outputs[0], skip_special_tokens=True))

In [9]:
%%time
device = 'cuda' if torch.cuda.is_available() else 'cpu'

predict_result(prompt)

<|im_start|> system
Bạn là một trợ lí AI hữu ích. Hãy trả lời người dùng một cách chính xác.
 
<|im_start|> user
Viết một hàm tính tổng hai số trong python
 
<|im_start|> assistant
Dưới đây là một hàm Python đơn giản nhận hai số làm đầu vào và trả về tổng của chúng:

```python
def add_two_numbers(a, b):
    return a + b

# Kiểm tra hàm
result = add_two_numbers(3, 5)
print(result)  # Kết quả: 8
```

Trong đoạn code này, chúng ta định nghĩa một hàm có tên là `add_two_numbers` nhận hai tham số: `a` và `b`. Bên trong hàm, chúng ta sử dụng toán tử `+` để cộng hai số lại với nhau, sau đó trả về kết quả.

Để kiểm tra hàm, chúng ta gọi nó với các số 3 và 5 làm đối số, và lưu kết quả vào biến `result`. Cuối cùng, chúng ta in ra giá trị của `result`, trong trường hợp này sẽ là 8.
CPU times: user 27.5 s, sys: 1.04 s, total: 28.5 s
Wall time: 33.5 s


In [10]:
prompt = '''
<|im_start|>system
Bạn là một trợ lí AI hữu ích. Hãy trả lời người dùng một cách chính xác.
<|im_end|>
<|im_start|>user
Hãy chỉ tôi ngồi lên một chiếc ghế
<|im_end|>
<|im_start|>assistant
'''.strip()

predict_result(prompt)

<|im_start|> system
Bạn là một trợ lí AI hữu ích. Hãy trả lời người dùng một cách chính xác.
 
<|im_start|> user
Hãy chỉ tôi ngồi lên một chiếc ghế
 
<|im_start|> assistant
Để ngồi lên một chiếc ghế, hãy làm theo các bước sau:

1. Đứng trước chiếc ghế: Tìm một chiếc ghế và di chuyển đến phía trước của nó. Đảm bảo rằng bạn có đủ không gian phía trước để có thể ngồi thoải mái.

2. Đặt mông của bạn lên ghế: Từ từ hạ mông của bạn xuống ghế, đảm bảo rằng bạn giữ tư thế đúng và ngồi thẳng lưng.

3. Đặt chân của bạn lên ghế: Đặt cả hai chân của bạn trên ghế, giữ chúng gần nhau hoặc cách nhau bằng chiều rộng của hông, tùy thuộc vào độ thoải mái của bạn. Đảm bảo rằng bạn giữ tư thế đúng và ngồi thẳng lưng.

4. Điều chỉnh ghế: Nếu cần, điều chỉnh chiều cao của ghế bằng cách sử dụng tay cầm hoặc cơ chế điều chỉnh tích hợp. Đảm bảo rằng cằm của bạn ở cùng mức với đầu gối của bạn.

5. Ngồi thoải mái: Khi ghế đã ở vị trí phù


# Fine-tuning LLM
## Download Dataset

In [11]:
data = load_dataset('alespalla/chatbot_instruction_prompts')
data

Generating train split:   0%|          | 0/258042 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/64511 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['response', 'prompt'],
        num_rows: 258042
    })
    test: Dataset({
        features: ['response', 'prompt'],
        num_rows: 64511
    })
})

In [12]:
type(data)

datasets.dataset_dict.DatasetDict

In [13]:
data['train'][2]

{'response': 'Bonjour',
 'prompt': 'Translate the phrase "Good Morning" to French '}

## Prepare training dataset

In [14]:
def generate_prompt(data_point):
    translator = Translator()
    vn_prompt = translator.translate(data_point['prompt'], src='en', dest='vi').text
    vn_response = translator.translate(data_point['response'], src='en', dest='vi').text

    return f'''
    <|im_start|>system
    Bạn là một trợ lí AI hữu ích. Hãy trả lời người dùng một cách chính xác.
    <|im_end|>
    <|im_start|>user
    {vn_prompt}
    <|im_end|>
    <|im_start|>assistant
    {vn_response}
    '''.strip()

def generate_and_tokenize_prompt(data_point):
    full_prompt = generate_prompt(data_point)
    tokenized_full_prompt = tokenizer(full_prompt, padding=True, truncation=True)
    return tokenized_full_prompt

In [15]:
data_train = data['train'].shard(num_shards=50, index=0).filter(lambda sample: sample['response'] != '' and sample['prompt'] != '').shuffle().map(generate_and_tokenize_prompt)

Filter:   0%|          | 0/5161 [00:00<?, ? examples/s]

Map:   0%|          | 0/5161 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


# Training

In [16]:
training_args = transformers.TrainingArguments(
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    num_train_epochs=1,
    learning_rate=2e-4,
    fp16=True,
    save_total_limit=3,
    logging_steps=1,
    output_dir='experiments',
    optim='paged_adamw_8bit',
    lr_scheduler_type='cosine',
    warmup_ratio=0.05
)

trainer = transformers.Trainer(
    model=model,
    train_dataset=data_train,
    args=training_args,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)

model.config.use_cache = False
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
1,4.467900
2,2.424800
3,4.398900
4,2.796700
5,3.584100
6,4.637900
7,3.905700
8,3.825000
9,2.974600
10,2.886500


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


TrainOutput(global_step=1290, training_loss=1.2086314336736073, metrics={'train_runtime': 8852.9306, 'train_samples_per_second': 0.583, 'train_steps_per_second': 0.146, 'total_flos': 3.088857922609152e+16, 'train_loss': 1.2086314336736073, 'epoch': 0.9998062391009495})

# Test Prediction

In [35]:
prompt = """
<|im_start|>system
Bạn là một trợ lí AI hữu ích. Hãy trả lời người dùng một cách chính xác.
<|im_end|>
<|im_start|>user
Mô tả về thành phố Pleiku
<|im_end|>
<|im_start|>assistant
""".strip()

predict_result(prompt)

<|im_start|> system
Bạn là một trợ lí AI hữu ích. Hãy trả lời người dùng một cách chính xác.
 
<|im_start|> user
Mô tả về thành phố Pleiku
 
<|im_start|> assistant
Gần đây, tôi đã đến thăm thành phố Pleiku và có một trải nghiệm tuyệt vời. Thành phố này có một bầu không khí thanh bình và yên bình, với những con đường và công viên đẹp đẽ, cùng những người dân thân thiện và thân thiện. Các tòa nhà lịch sử của thành phố và các di tích văn hóa đã tạo ra một bầu không khí độc đáo và hấp dẫn, khiến nó trở thành một điểm đến phải ghé thăm đối với những người đam mê văn hóa. Ngoài ra, các món ăn địa phương cũng rất ngon và đa dạng, khiến thành phố trở thành một điểm đến ẩm thực tuyệt vời. Nhìn chung, thành phố này là một điểm đến tuyệt vời để tham quan và khám phá.


# Save model to huggingface

In [ ]:
!huggingface-cli login

In [ ]:
!pip install huggingface_hub
from huggingface_hub import notebook_login

notebook_login()

In [19]:
model.save_pretrained('trained-model')

In [28]:
PEFT_MODEL = 'toan-ly/vinallama-peft-7b-chatbot'

model.push_to_hub(
    PEFT_MODEL, use_auth_token=True
)

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:834: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


adapter_model.safetensors:   0%|          | 0.00/160M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/toan-ly/vinallama-peft-7b-chatbot/commit/1be69a56bf6aa6653a4ddc88a5a0f7adb477153e', commit_message='Upload model', commit_description='', oid='1be69a56bf6aa6653a4ddc88a5a0f7adb477153e', pr_url=None, pr_revision=None, pr_num=None)

# Inference

In [30]:
config = PeftConfig.from_pretrained(PEFT_MODEL)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    return_dict=True,
    quantization_config=bnb_config,
    device_map='auto',
    trust_remote_code=True
)

tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
tokenizer.pad_token = tokenizer.eos_token

model = PeftModel.from_pretrained(model, PEFT_MODEL)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


adapter_model.safetensors:   0%|          | 0.00/160M [00:00<?, ?B/s]

In [31]:
generation_config = model.generation_config
generation_config.max_new_tokens = 200
generation_config.temperature = 0.7
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

In [36]:
%%time
device = 'cuda' if torch.cuda.is_available() else 'cpu'

prompt = """
<|im_start|>system
Bạn là một trợ lí AI hữu ích. Hãy trả lời người dùng một cách chính xác.
<|im_end|>
<|im_start|>user
Các bước làm món bún đậu mắm tôm
<|im_end|>
<|im_start|>assistant
""".strip()

predict_result(prompt)

<|im_start|> system
Bạn là một trợ lí AI hữu ích. Hãy trả lời người dùng một cách chính xác.
 
<|im_start|> user
Các bước làm món bún đậu mắm tôm
 
<|im_start|> assistant
1. Bắt đầu bằng cách đun sôi một nồi nước lớn với một ít muối.
2. Cho đậu nành vào nước sôi và nấu cho đến khi mềm.
3. Lấy đậu nành ra khỏi nước sôi và để riêng.
4. Trong một tô lớn, trộn mắm tôm, rau mùi, nước mắm, tỏi và ớt.
5. Cho đậu nành vào và khuấy đều.
6. Đun nóng hỗn hợp trong lò vi sóng hoặc trên lửa vừa cho đến khi nóng đều.
7. Phục vụ nóng và thưởng thức. Thưởng thức!
CPU times: user 17.1 s, sys: 261 ms, total: 17.4 s
Wall time: 18.3 s
